In [1]:
%load_ext autotime

In [2]:
import sys
#sys.path.append('/scratch/pradap/python-work/enrique/')
sys.path.append('/Users/pradap/Documents/Research/Python-Package/enrique/')

time: 1 ms


In [3]:
import magellan as mg

DEBUG:Cloud:Log file (/Users/pradap/.picloud/cloud.log) opened


time: 1.77 s


In [4]:
import pandas as pd
import numpy as np

time: 941 µs


In [5]:
mg.init_jvm()

True

time: 102 ms


In [6]:
wal =  mg.read_csv(mg.get_install_path() + '/datasets/books/walmart.csv',
                    dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                    low_memory=False, key='id')


time: 1.16 s


In [7]:
bwk = mg.read_csv(mg.get_install_path() + '/datasets/books/bowker.csv', 
                  dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                  low_memory=False, key='id')


time: 86.7 ms


In [8]:
ab = mg.AttrEquivalenceBlocker()
C = ab.block_tables(wal, bwk, 'isbn', 'isbn', ['title', 'author'], ['title', 'author'])


time: 194 ms


In [9]:
L = mg.read_csv('label_ab_correct_books.csv', ltable=wal, rtable=bwk)


time: 2.93 ms


In [10]:
feat_table = mg.get_features_for_matching(wal, bwk)
f = feat_table.ix[[3,7,18,26, 53]]

time: 3.1 s


In [11]:
G = mg.extract_feat_vecs(L, feat_table=f, attrs_after='gold')

time: 350 ms


In [12]:
dt = mg.DTMatcher(random_state=100)
svm = mg.SVMMatcher()
rf = mg.RFMatcher()
nb = mg.NBMatcher()
lg = mg.LogRegMatcher()

time: 3.14 ms


In [13]:
# impute values
G.fillna(0, inplace=True)

time: 1.9 ms


In [14]:
result = mg.select_matcher([dt, rf, svm, nb, lg], table=G, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
                                  target_attr='gold', metric='precision')

time: 121 ms


In [15]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree_65818645721942595441,<magellan.matcher.dtmatcher.DTMatcher object a...,5,0.875000,0.777778,0.909091,0.916667,0.833333,0.862374
1,RandomForest_08693065721942595441,<magellan.matcher.rfmatcher.RFMatcher object a...,5,0.875000,0.750000,0.909091,1.000000,0.833333,0.873485
2,SVM_48112455721942595441,<magellan.matcher.svmmatcher.SVMMatcher object...,5,0.500000,0.750000,0.888889,0.900000,0.833333,0.774444
3,NaiveBayes_69219765721942595441,<magellan.matcher.nbmatcher.NBMatcher object a...,5,0.875000,0.666667,0.875000,0.900000,1.000000,0.863333
4,LogisticRegression_40784475721942595441,<magellan.matcher.logregmatcher.LogRegMatcher ...,5,0.636364,0.727273,0.800000,0.900000,0.833333,0.779394


time: 13.7 ms


In [16]:
#d = mg.train_test_split(G, train_proportion=0.7)
train = mg.read_csv('train.csv', ltable=wal, rtable=bwk)
test = mg.read_csv('test.csv', ltable=wal, rtable=bwk)


time: 9.22 ms


In [17]:
dt.fit(table=train, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'], target_attr='gold')

time: 8.77 ms


In [18]:
#mg.vis_debug_dt(dt, train, test, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
#                                      target_attr='gold')

IndentationError: unexpected indent (<ipython-input-18-b8b2c5b917a8>, line 2)

In [19]:
P = dt.predict(table=test, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'], target_attr='predicted', append=True, inplace=False)

time: 8.07 ms


In [20]:
P

,_id,ltable.id,rtable.id,title_title_jac_qgm_3_qgm_3,author_author_jac_qgm_3_qgm_3,binding_binding_jac_qgm_3_qgm_3,publisher_publisher_jac_qgm_3_qgm_3,lang_lang_lev,gold,predicted
0,29,3618742,9780060837563,1.000000,1.000000,1.000000,0.320000,1,1,1
1,43,14028323,9780217618472,0.421053,1.000000,1.000000,1.000000,1,0,1
2,61,15341050,9780217227834,0.792453,0.666667,1.000000,1.000000,1,1,1
3,31,10865895,9780153234514,0.204819,0.000000,1.000000,1.000000,1,0,0
4,53,10083762,9780142412145,0.861111,0.264706,0.368421,0.406250,1,1,1
5,19,7809485,9780152062279,0.739130,1.000000,1.000000,0.547619,1,1,1
6,65,15373263,9780217300049,0.969697,1.000000,1.000000,1.000000,1,1,1
7,64,15359574,9780217298445,0.936170,1.000000,1.000000,1.000000,1,1,1
8,21,4763339,9780205327270,0.957447,0.277778,0.000000,0.000000,1,0,1
9,37,15368811,9780217110952,0.852941,1.000000,1.000000,1.000000,1,1,1


time: 43.3 ms


In [21]:
trigger = mg.MatchTrigger()
trigger.add_cond_rule('title_title_jac_qgm_3_qgm_3(ltuple, rtuple) > 0.9', f)

True

time: 2.84 ms


In [22]:
trigger.add_cond_status(True)
trigger.add_action(1)

True

time: 2.77 ms


In [27]:
for k, t in P.iterrows():
    p = t

time: 1.94 ms


In [28]:
p

_id                                    5.600000e+01
ltable.id                              1.343398e+07
rtable.id                              9.780206e+12
title_title_jac_qgm_3_qgm_3            4.117647e-01
author_author_jac_qgm_3_qgm_3          2.631579e-01
binding_binding_jac_qgm_3_qgm_3        0.000000e+00
publisher_publisher_jac_qgm_3_qgm_3    7.333333e-01
lang_lang_lev                          1.000000e+00
gold                                   1.000000e+00
predicted                              1.000000e+00
Name: 20, dtype: float64

time: 3.58 ms


In [23]:
trigger.execute(P, 'predicted')

/Users/pradap/anaconda/lib/python2.7/site-packages/pandas/core/index.py:805: FutureWarning: scalar indexers for index type Int64Index should be integers and not floating point
  type(self).__name__),FutureWarning)


KeyError: 31.0

time: 716 ms
